In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir(os.path.join(os.getcwd(), "..", "data"));

In [3]:
target_gw = 8;

In [4]:
target_season = "2019";

In [5]:
goalies = [
    {
        "player": "David de Gea",
        "opponent": "Newcastle United",
        "home": 0,
        "pos": "GK",
    },
    {
        "player": "David Button",
        "opponent": "Tottenham Hotspur",
        "home": 1,
        "pos": "GK",
    },
]

In [6]:
defenders = [
    {
        "player": "Trent Alexander-Arnold",
        "opponent": "Leicester City",
        "home": 1,
        "pos": "DEF",
    },
    {
        "player": "Nicolás Otamendi",
        "opponent": "Wolverhampton Wanderers",
        "home": 1,
        "pos": "DEF",
    },
    {
        "player": "Issa Diop",
        "opponent": "Crystal Palace",
        "home": 1,
        "pos": "DEF",
    },
    {
        "player": "Martin Kelly",
        "opponent": "West Ham United",
        "home": 0,
        "pos": "DEF",
    },
    {
        "player": "John Lundstram",
        "opponent": "Watford",
        "home": 0,
        "pos": "DEF",
    },
]

In [7]:
midfields = [
    {
        "player": "Sadio Mané",
        "opponent": "Leicester City",
        "home": 1,
        "pos": "MID",
    },
    {
        "player": "Harry Wilson",
        "opponent": "Arsenal",
        "home": 0,
        "pos": "MID",
    },
    {
        "player": "Mason Mount",
        "opponent": "Southampton",
        "home": 0,
        "pos": "MID",
    },
    {
        "player": "Kevin De Bruyne",
        "opponent": "Wolverhampton Wanderers",
        "home": 1,
        "pos": "MID",
    },
    {
        "player": "Todd Cantwell",
        "opponent": "Aston Villa",
        "home": 1,
        "pos": "MID",
    },
]

In [8]:
forwards = [
    {
        "player": "Tammy Abraham",
        "opponent": "Southampton",
        "home": 0,
        "pos": "FWD",
    },
    {
        "player": "Teemu Pukki",
        "opponent": "Aston Villa",
        "home": 1,
        "pos": "FWD",
    },
    {
        "player": "Pierre-Emerick Aubameyang",
        "opponent": "Bournemouth",
        "home": 1,
        "pos": "FWD",
    },
]

In [9]:
data = list(np.concatenate((goalies, defenders, midfields, forwards)))

In [10]:
df = pd.read_csv("{}-prepared-data.csv".format(target_season))

In [11]:
df.sort_values(["season", "GW"], inplace=True)

In [12]:
df.head()

,season,GW,at_home,player_id,opponent_id,champion_ls,top5_ls,bottom5_ls,promoted_ts,assists,...,result,result_pgw,saves,saves_pgw,threat,threat_pgw,yellow_cards,yellow_cards_pgw,total_points_pgw,total_points
0,2019,1,1,1,10,1,1,0,0,0,...,-1,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0
7,2019,1,1,2,13,0,0,1,0,0,...,1,0.0,0,0.0,0.0,0.0,0,0.0,0.0,1
14,2019,1,1,3,29,0,0,0,1,0,...,0,0.0,2,0.0,0.0,0.0,0,0.0,0.0,2
21,2019,1,1,5,4,0,1,0,0,0,...,1,0.0,0,0.0,2.0,0.0,0,0.0,0.0,8
28,2019,1,1,6,21,0,0,1,0,0,...,-1,0.0,0,0.0,2.0,0.0,0,0.0,0.0,0


In [13]:
player_id_df = pd.read_csv("player_id_map.csv");

In [14]:
team_id_df = pd.read_csv("team_id_map.csv");

In [15]:
players = [];
teams = [];
at_homes = [];
positions = [];

In [16]:
team_ids = [];
player_ids = [];

champion_ls = [];
top5_ls = [];
bottom5_ls = [];
promoted_ts = [];

In [17]:
for inst in data:
    players.append(inst["player"]);
    positions.append(inst["pos"]);
    teams.append(inst["opponent"]);
    at_homes.append(inst["home"]);

In [18]:
for team in teams:
    team_ids.append(team_id_df[team_id_df["team_name"] == team]["team_id"].iloc[0]);

In [19]:
for player in players:
    player_ids.append(player_id_df[player_id_df["actual_name"] == player]["player_id"].iloc[0]);

In [20]:
for team_id in team_ids:
    champion_ls.append(df[df["opponent_id"] == team_id]["champion_ls"].iloc[0]);
    top5_ls.append(df[df["opponent_id"] == team_id]["top5_ls"].iloc[0]);
    bottom5_ls.append(df[df["opponent_id"] == team_id]["bottom5_ls"].iloc[0]);
    promoted_ts.append(df[df["opponent_id"] == team_id]["promoted_ts"].iloc[0]);

In [21]:
ndf = pd.DataFrame(data={"player_id": player_ids, "position": positions, "opponent_id": team_ids, "at_home": at_homes, 
                         "champion_ls": champion_ls, "top5_ls": top5_ls, "bottom5_ls": bottom5_ls, 
                         "promoted_ts": promoted_ts})

In [22]:
cols = ['assists', 'bonus',
       'bps', 'clean_sheets',
        'creativity',
       'goals_conceded', 'goals_scored', 
       'ict_index', 'influence', 'minutes',
       'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards',
       'saves', 
       'threat','yellow_cards', "result", 'total_points']

In [23]:
results = []
for col in cols:
    print("Performing act on column: {}".format(col))
    for player in player_ids:
        col_to_find = col
        results.append(df[(df["player_id"] == player) & (df["GW"] == target_gw-1)][col_to_find].iloc[0])
    ndf[col + "_pgw"] = results
    results = []

Performing act on column: assists
Performing act on column: bonus
Performing act on column: bps
Performing act on column: clean_sheets
Performing act on column: creativity
Performing act on column: goals_conceded
Performing act on column: goals_scored
Performing act on column: ict_index
Performing act on column: influence
Performing act on column: minutes
Performing act on column: own_goals
Performing act on column: penalties_missed
Performing act on column: penalties_saved
Performing act on column: red_cards
Performing act on column: saves
Performing act on column: threat
Performing act on column: yellow_cards
Performing act on column: result
Performing act on column: total_points


In [24]:
main_df = pd.read_csv("combined-season-data-rf.csv");

In [25]:
main_df.columns

Index(['season', 'GW', 'at_home', 'player_id', 'opponent_id', 'champion_ls',
       'top5_ls', 'bottom5_ls', 'promoted_ts', 'assists', 'assists_pgw',
       'assists_rlf', 'assists_rsf', 'bonus', 'bonus_pgw', 'bonus_rlf',
       'bonus_rsf', 'bps', 'bps_pgw', 'bps_rlf', 'bps_rsf', 'clean_sheets',
       'clean_sheets_pgw', 'clean_sheets_rlf', 'clean_sheets_rsf',
       'creativity', 'creativity_pgw', 'creativity_rlf', 'creativity_rsf',
       'goals_conceded', 'goals_conceded_pgw', 'goals_conceded_rlf',
       'goals_conceded_rsf', 'goals_scored', 'goals_scored_pgw',
       'goals_scored_rlf', 'goals_scored_rsf', 'ict_index', 'ict_index_pgw',
       'ict_index_rlf', 'ict_index_rsf', 'influence', 'influence_pgw',
       'influence_rlf', 'influence_rsf', 'minutes', 'minutes_pgw',
       'minutes_rlf', 'minutes_rsf', 'opponent_form', 'own_goals',
       'own_goals_pgw', 'own_goals_rlf', 'own_goals_rsf', 'penalties_missed',
       'penalties_missed_pgw', 'penalties_missed_rlf', 'penalties_

In [26]:
results = []
for col in cols:
    col_to_find = col + "_rlf";
    
    print("Performing act on column: {}".format(col))
    
    for player in player_ids:
        opponent_id = ndf[ndf["player_id"] == player]["opponent_id"].iloc[0]
        
        sub_df = main_df[(main_df["player_id"] == player) & (main_df["opponent_id"] == opponent_id)];
        
        if (sub_df.shape[0] == 0):
            results.append(0);
        else:
            index = list(sub_df.index).pop();
            results.append(sub_df["result_rlf"].loc[index]);
            
    ndf[col_to_find] = results
    results = []

Performing act on column: assists
Performing act on column: bonus
Performing act on column: bps
Performing act on column: clean_sheets
Performing act on column: creativity
Performing act on column: goals_conceded
Performing act on column: goals_scored
Performing act on column: ict_index
Performing act on column: influence
Performing act on column: minutes
Performing act on column: own_goals
Performing act on column: penalties_missed
Performing act on column: penalties_saved
Performing act on column: red_cards
Performing act on column: saves
Performing act on column: threat
Performing act on column: yellow_cards
Performing act on column: result
Performing act on column: total_points


In [27]:
results = []
for col in cols:
    col_to_find = col + "_rsf";
    
    print("Performing act on column: {}".format(col))
    
    for player in player_ids:
        opponent_id = ndf[ndf["player_id"] == player]["opponent_id"].iloc[0]
        at_home = ndf[ndf["player_id"] == player]["at_home"].iloc[0]
        
        sub_df = main_df[(main_df["player_id"] == player) & (main_df["opponent_id"] == opponent_id) & (main_df["at_home"] == at_home)];
        
        if (sub_df.shape[0] == 0):
            results.append(0);
        else:
            index = list(sub_df.index).pop();
            results.append(sub_df["result_rlf"].loc[index]);
            
    ndf[col_to_find] = results
    results = []

Performing act on column: assists
Performing act on column: bonus
Performing act on column: bps
Performing act on column: clean_sheets
Performing act on column: creativity
Performing act on column: goals_conceded
Performing act on column: goals_scored
Performing act on column: ict_index
Performing act on column: influence
Performing act on column: minutes
Performing act on column: own_goals
Performing act on column: penalties_missed
Performing act on column: penalties_saved
Performing act on column: red_cards
Performing act on column: saves
Performing act on column: threat
Performing act on column: yellow_cards
Performing act on column: result
Performing act on column: total_points


In [28]:
player_gws = 3;

In [29]:
player_forms = [];
for player in player_ids:
    sub_df = main_df[main_df["player_id"] == player].tail(player_gws);
    indexes = list(sub_df.index)
    
    avg_points_player = np.mean(sub_df["total_points"])
    points_overall = [];
            
    for ind, row in sub_df.iterrows():
        season = row["season"];
        gw = row["GW"];
        avg_point_ov = np.mean(main_df[(main_df["season"] == season) & (main_df["GW"] == gw)]["total_points"])
        points_overall.append(avg_point_ov);
            
    avg_points_overall = np.mean(points_overall);
    player_forms.append((avg_points_player / avg_points_overall));
                
    print("Done player {}".format(player))
ndf["player_form"] = player_forms

Done player 176
Done player 947
Done player 648
Done player 514
Done player 977
Done player 451
Done player 1155
Done player 585
Done player 267
Done player 1178
Done player 385
Done player 1208
Done player 882
Done player 1206
Done player 858


In [30]:
team_gws = 4;

In [31]:
sub_df = main_df[main_df["opponent_id"] == 1].tail(team_gws);

In [32]:
opponent_forms = [];
for team in team_ids:
    gws_to_look = np.arange((target_gw-team_gws), target_gw);
    results = [];
    for gw_ in gws_to_look:
        result_gw = np.unique(main_df[(main_df["opponent_id"] == team) & (main_df["season"] == int(target_season)) & 
                                 (main_df["GW"] == gw_)]["result"])
        for res in result_gw:
            results.append(-res);
    opponent_forms.append(np.mean(results));
    print("Done team {}".format(team))
ndf["opponent_form"] = opponent_forms


Done team 23
Done team 17
Done team 8
Done team 26
Done team 5
Done team 20
Done team 18
Done team 8
Done team 1
Done team 13
Done team 26
Done team 27
Done team 13
Done team 27
Done team 2


In [33]:
ndf

,player_id,position,opponent_id,at_home,champion_ls,top5_ls,bottom5_ls,promoted_ts,assists_pgw,bonus_pgw,...,penalties_missed_rsf,penalties_saved_rsf,red_cards_rsf,saves_rsf,threat_rsf,yellow_cards_rsf,result_rsf,total_points_rsf,player_form,opponent_form
0,176,GK,23,0,0,0,1,0,0,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.316759,-0.50
1,947,GK,17,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25
2,648,DEF,8,1,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.316759,0.50
3,514,DEF,26,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.758994,-0.25
4,977,DEF,5,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.653408,0.25
5,451,DEF,20,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.326704,0.50
6,1155,DEF,18,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.537877,-0.50
7,585,MID,8,1,0,0,0,0,0,0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.422346,0.50
8,267,MID,1,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.990056,0.25
9,1178,MID,13,0,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.758994,-0.25


In [34]:
ndf.columns

Index(['player_id', 'position', 'opponent_id', 'at_home', 'champion_ls',
       'top5_ls', 'bottom5_ls', 'promoted_ts', 'assists_pgw', 'bonus_pgw',
       'bps_pgw', 'clean_sheets_pgw', 'creativity_pgw', 'goals_conceded_pgw',
       'goals_scored_pgw', 'ict_index_pgw', 'influence_pgw', 'minutes_pgw',
       'own_goals_pgw', 'penalties_missed_pgw', 'penalties_saved_pgw',
       'red_cards_pgw', 'saves_pgw', 'threat_pgw', 'yellow_cards_pgw',
       'result_pgw', 'total_points_pgw', 'assists_rlf', 'bonus_rlf', 'bps_rlf',
       'clean_sheets_rlf', 'creativity_rlf', 'goals_conceded_rlf',
       'goals_scored_rlf', 'ict_index_rlf', 'influence_rlf', 'minutes_rlf',
       'own_goals_rlf', 'penalties_missed_rlf', 'penalties_saved_rlf',
       'red_cards_rlf', 'saves_rlf', 'threat_rlf', 'yellow_cards_rlf',
       'result_rlf', 'total_points_rlf', 'assists_rsf', 'bonus_rsf', 'bps_rsf',
       'clean_sheets_rsf', 'creativity_rsf', 'goals_conceded_rsf',
       'goals_scored_rsf', 'ict_index_rsf', 

In [35]:
ndf.to_csv("gw{}-fpl-data.csv".format(target_gw), index=False)